In [3]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller 
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [4]:
file_path = r"C:\Users\Madnessity\Desktop\teva_sales_small_normalized_slim.csv"
data = pd.read_csv(file_path)
data.rename(columns={'diff':'diff_1'}, inplace=True)

# Choosing one molecule to start with
target_molecule = 0.315789473684211
m_data = data[abs(data['inn_normalized'] - target_molecule) < 1e-10].copy()

# type(m_data)
# data['inn_normalized'].unique
# result 1730 rows × 70 columns

In [7]:
m_data = m_data.drop(['Unnamed: 0', 'sm_sales'], axis=1)
m_data

,morion_id,month,atc_code_5_normalized,brand_normalized,inn_normalized,nfc_1_normalized,strength_unit_normalized,drug_size_unit_normalized,form_normalized,diff_1,strength_amount_normalized,pack_size_normalized,drug_size_amount_normalized
0,1479,2019-02-01,0.111111,0.223242,0.315789,0.375,0.4,1.0,1.0,-584.098000,0.0025,0.411765,0.0
1,1479,2019-03-01,0.111111,0.223242,0.315789,0.375,0.4,1.0,1.0,470.293600,0.0025,0.411765,0.0
2,1479,2019-04-01,0.111111,0.223242,0.315789,0.375,0.4,1.0,1.0,-50.617120,0.0025,0.411765,0.0
3,1479,2019-05-01,0.111111,0.223242,0.315789,0.375,0.4,1.0,1.0,-146.281696,0.0025,0.411765,0.0
4,1479,2019-06-01,0.111111,0.223242,0.315789,0.375,0.4,1.0,1.0,-3272.007357,0.0025,0.411765,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63361,614194,2023-08-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,-16.106641,0.0025,0.495798,0.0
63362,614194,2023-09-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,-12.941313,0.0025,0.495798,0.0
63363,614194,2023-10-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,5.770950,0.0025,0.495798,0.0
63364,614194,2023-11-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,-13.095240,0.0025,0.495798,0.0


In [9]:
# m_data.to_csv(r"C:\Users\Madnessity\Desktop\temp.csv")

### Choose the most sold product by morion ID

In [12]:
morion_diff_sum = m_data.groupby('morion_id')['diff_1'].sum()
max_diff_morion_id = morion_diff_sum.idxmax()
max_diff_sum = morion_diff_sum.max()
target_morion_id = max_diff_morion_id
target_morion_id

112504

### Morion IDs into columns

In [20]:
df_transposed_drugs = m_data.pivot(index='month', columns='morion_id', values='diff_1')
to_be_renamed = {}
for col in df_transposed_drugs.columns:
    to_be_renamed[col] = f'morion_{col}_diff_1'
df_transposed_drugs = df_transposed_drugs.rename(columns=to_be_renamed)



In [22]:
df_transposed_drugs

morion_id,morion_1479_diff_1,morion_20731_diff_1,morion_71569_diff_1,morion_71570_diff_1,morion_74821_diff_1,morion_75666_diff_1,morion_75912_diff_1,morion_75913_diff_1,morion_75916_diff_1,morion_75917_diff_1,...,morion_328378_diff_1,morion_378561_diff_1,morion_378563_diff_1,morion_378565_diff_1,morion_444531_diff_1,morion_444532_diff_1,morion_444535_diff_1,morion_444536_diff_1,morion_614190_diff_1,morion_614194_diff_1
month,,,,,,,,,,,,,,,,,,,,,
2019-02-01,-584.098000,-178.624000,-23.440000,-164.302000,-255.422000,-357.480000,-60.360000,-20.706000,-155.262000,-76.632000,...,387.320000,-1.648000,-1.148000,-1.156000,-284.052000,545.258000,73.952000,328.546000,-2.388000,-3.992000
2019-03-01,470.293600,42.778800,17.562000,-85.379600,-251.465600,638.136000,-16.928000,-22.968800,-62.573600,-24.197600,...,220.514000,-1.318400,-0.918400,-0.924800,-26.217600,914.402400,-212.266400,349.360800,-1.910400,-3.193600
2019-04-01,-50.617120,-53.610960,-46.474400,-167.157680,-283.972480,-44.419200,-84.448400,-18.439040,-135.680880,-79.194080,...,-176.972800,-1.054720,-0.734720,-0.739840,889.575920,209.983920,367.846880,4.030640,-1.528320,-2.554880
2019-05-01,-146.281696,-5.656768,10.652480,-25.328144,-223.561984,80.206640,-95.932720,-16.575232,-99.786704,-42.377264,...,-12.088240,-0.843776,-0.587776,-0.591872,1385.292736,-207.256864,1094.277504,-248.311488,-1.222656,-2.043904
2019-06-01,-3272.007357,115.968586,-30.390016,2648.281485,-201.737587,165.907312,-114.890176,-29.676186,-229.751363,-95.177811,...,-161.248592,-0.675021,-0.470221,-0.473498,-10.593811,-441.325491,541.176003,-531.961190,-0.978125,-1.635123
2019-07-01,-5651.013885,1032.316868,257.349987,2761.989188,-166.630070,2089.245850,-71.066141,-31.886948,-82.561091,-72.742249,...,-153.010874,-0.540017,-0.376177,-0.378798,1849.880951,74.623607,347.018803,17.101048,-0.782500,-1.308099
2019-08-01,-5162.207108,1691.249495,569.879990,361.233350,-143.446056,4743.836680,-55.398913,-25.627559,-21.732872,-88.389799,...,78.657301,-0.432013,-0.300941,-0.303038,355.266761,-47.199114,292.203042,-100.461162,-0.626000,-1.046479
2019-09-01,-1913.071687,535.671596,624.993992,261.794680,-77.226845,3637.661344,-35.533130,10.673953,21.141702,-35.067839,...,117.059841,-0.345611,-0.240753,-0.242431,794.665409,48.454709,103.466434,-37.928929,-0.500800,-0.837183
2019-10-01,3607.004651,105.297277,51.069193,-1896.230256,-83.079476,1439.301075,-5.074504,-10.082838,-4.886638,14.195729,...,285.957873,-0.276489,-0.192602,0.962055,3279.384327,21.087767,1579.123147,171.948856,-0.400640,-0.669746


In [24]:
selected_drug_df = m_data[data['morion_id'] == max_diff_morion_id].copy()
selected_drug_df

C:\Users\Madnessity\AppData\Local\Temp\ipykernel_358572\2297536780.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_drug_df = m_data[data['morion_id'] == max_diff_morion_id].copy()


,morion_id,month,atc_code_5_normalized,brand_normalized,inn_normalized,nfc_1_normalized,strength_unit_normalized,drug_size_unit_normalized,form_normalized,diff_1,strength_amount_normalized,pack_size_normalized,drug_size_amount_normalized
19588,112504,2019-02-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,-405.120000,0.0025,0.243697,0.0
19589,112504,2019-03-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,3264.228000,0.0025,0.243697,0.0
19590,112504,2019-04-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,4773.586400,0.0025,0.243697,0.0
19591,112504,2019-05-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,7653.625120,0.0025,0.243697,0.0
19592,112504,2019-06-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,3995.190096,0.0025,0.243697,0.0
19593,112504,2019-07-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,5146.636077,0.0025,0.243697,0.0
19594,112504,2019-08-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,4377.714861,0.0025,0.243697,0.0
19595,112504,2019-09-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,2895.499889,0.0025,0.243697,0.0
19596,112504,2019-10-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,5435.313911,0.0025,0.243697,0.0
19597,112504,2019-11-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,6253.157129,0.0025,0.243697,0.0


### Merge the tables

In [26]:
# Merging the dataframes on 'morion_id'
merged_df = pd.merge(selected_drug_df, df_transposed_drugs, on='month')
merged_df = merged_df.drop(['morion_id', 'diff_1'], axis=1)

target_morion_id


112504

In [28]:
merged_df[f'morion_{target_morion_id}_diff_1']

0      -405.120000
1      3264.228000
2      4773.586400
3      7653.625120
4      3995.190096
5      5146.636077
6      4377.714861
7      2895.499889
8      5435.313911
9      6253.157129
10     6728.451703
11     3867.979363
12     6095.921490
13     3061.767192
14    -7256.208246
15    -3248.424597
16    -1547.495678
17     -185.028542
18     -453.642834
19     1771.675733
20     2980.442586
21     2970.160069
22     5939.746055
23     2134.308844
24     2983.157075
25     6597.795660
26     2180.174528
27     1172.387623
28     1383.786098
29     -845.851122
30    -2321.724897
31     -266.135918
32      997.421266
33     -360.610987
34      315.917210
35      838.791768
36     4596.233414
37    10352.786732
38    -6289.834615
39     -693.173692
40    -5727.916953
41    -6440.831563
42    -3728.727250
43      -23.691800
44     6792.022560
45     3656.196048
46     4321.900838
47        6.666671
48     1855.511337
49    12530.875069
50      801.456055
51     2270.256844
52      170.

In [30]:
merged_df

,month,atc_code_5_normalized,brand_normalized,inn_normalized,nfc_1_normalized,strength_unit_normalized,drug_size_unit_normalized,form_normalized,strength_amount_normalized,pack_size_normalized,...,morion_328378_diff_1,morion_378561_diff_1,morion_378563_diff_1,morion_378565_diff_1,morion_444531_diff_1,morion_444532_diff_1,morion_444535_diff_1,morion_444536_diff_1,morion_614190_diff_1,morion_614194_diff_1
0,2019-02-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,387.320000,-1.648000,-1.148000,-1.156000,-284.052000,545.258000,73.952000,328.546000,-2.388000,-3.992000
1,2019-03-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,220.514000,-1.318400,-0.918400,-0.924800,-26.217600,914.402400,-212.266400,349.360800,-1.910400,-3.193600
2,2019-04-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-176.972800,-1.054720,-0.734720,-0.739840,889.575920,209.983920,367.846880,4.030640,-1.528320,-2.554880
3,2019-05-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-12.088240,-0.843776,-0.587776,-0.591872,1385.292736,-207.256864,1094.277504,-248.311488,-1.222656,-2.043904
4,2019-06-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-161.248592,-0.675021,-0.470221,-0.473498,-10.593811,-441.325491,541.176003,-531.961190,-0.978125,-1.635123
5,2019-07-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-153.010874,-0.540017,-0.376177,-0.378798,1849.880951,74.623607,347.018803,17.101048,-0.782500,-1.308099
6,2019-08-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,78.657301,-0.432013,-0.300941,-0.303038,355.266761,-47.199114,292.203042,-100.461162,-0.626000,-1.046479
7,2019-09-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,117.059841,-0.345611,-0.240753,-0.242431,794.665409,48.454709,103.466434,-37.928929,-0.500800,-0.837183
8,2019-10-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,285.957873,-0.276489,-0.192602,0.962055,3279.384327,21.087767,1579.123147,171.948856,-0.400640,-0.669746
9,2019-11-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,437.584298,-0.221191,-0.154082,79.349644,4288.041462,86.506213,2044.490518,83.281085,-0.320512,-0.535797


### Functions

check_statio(df, col_name)

plot_partial_autocorr(one_drug_df, time_col_name, value_col_name)

calc_pacf_lags(df, col_name)

add_next_differencing_level(df, diff_level)

In [35]:
def check_statio(df, col_name):
    # Perform the Dickey-Fuller test 
    result = adfuller(df[col_name], autolag='BIC') 
    p_value = result[1] 
    print(f'ADF Statistic: {result[0]}') 
    print(f'p-value: {p_value}') 
    print('Stationary' if p_value < 0.05 else 'Non-Stationary')
    return True if p_value < 0.05 else False

# check_statio(m_data, 'diff_1' )

In [37]:
def plot_partial_autocorr(one_drug_df, time_col_name, value_col_name):
    # Preparation of the df
    sales_data=one_drug_df[[time_col_name, value_col_name]] 
    sales_data=pd.DataFrame(sales_data) 
    sales_data[time_col_name]=pd.to_datetime(sales_data[time_col_name]) 
    df1 = sales_data.set_index(time_col_name) 
    monthly_sales = df1.resample('M').mean() 

    # Plotting the Partial Autocorrelation Function (PACF)
    plt.figure(figsize=(10, 6))
    plot_pacf(monthly_sales) 
    plt.title('Partial Autocorrelation Function (PACF) for one drug')
    plt.xlabel('Lags')
    plt.ylabel('Autocorrelation')
    plt.grid(True)
    plt.show()
    return 0
# plot_partial_autocorr(m_data, 'month', 'sm_sales')

In [39]:
from statsmodels.tsa.stattools import pacf

def calc_pacf_lags(df, col_name):
    # Calculate PACF values
    pacf_values_diff = pacf(df[col_name])
    
    n = len(df)
    conf_interval = 1.96 / (n ** 0.5)
    significant_lags = []

    for i, value in enumerate(pacf_values_diff):
        if abs(value) > conf_interval:
            if significant_lags == [] or (i - significant_lags[-1] == 1):
                significant_lags.append(i)
            else:
                break

    return len(significant_lags)
# calc_pacf_lags(m_data, 'diff')

In [41]:
def add_next_differencing_level(df, morion, diff_level):
    # Calculate the difference in Sales for each Morion ID group
    df[f"morion_{morion}_diff_{diff_level}"] = df[f"morion_{morion}_diff_{diff_level-1}"].diff()
    return 0


### ACF and PACF

Step 7: Identify Model Parameters Identify the SARIMA model parameters (p, d, q, P, D, Q, s) using ACF and PACF plots.

ACF Plot: This function generates an ACF plot, which is a plot of autocorrelations of the differenced time series. Autocorrelation measures the relationship between a data point and previous data points at different lags. PACF Plot: This function generates a PACF plot, which is a plot of partial autocorrelations of the differenced time series. Partial autocorrelation represents the correlation between a data point and a lag while adjusting for the influence of other lags.

In [45]:
m_data = merged_df.copy()

check_statio(df, col_name) --> True/False

plot_partial_autocorr(one_drug_df, time_col_name, value_col_name) --> 0

calc_pacf_lags(df, col_name) --> int

add_next_differencing_level(df, diff_level) --> 0


In [49]:
m_data

,month,atc_code_5_normalized,brand_normalized,inn_normalized,nfc_1_normalized,strength_unit_normalized,drug_size_unit_normalized,form_normalized,strength_amount_normalized,pack_size_normalized,...,morion_328378_diff_1,morion_378561_diff_1,morion_378563_diff_1,morion_378565_diff_1,morion_444531_diff_1,morion_444532_diff_1,morion_444535_diff_1,morion_444536_diff_1,morion_614190_diff_1,morion_614194_diff_1
0,2019-02-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,387.320000,-1.648000,-1.148000,-1.156000,-284.052000,545.258000,73.952000,328.546000,-2.388000,-3.992000
1,2019-03-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,220.514000,-1.318400,-0.918400,-0.924800,-26.217600,914.402400,-212.266400,349.360800,-1.910400,-3.193600
2,2019-04-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-176.972800,-1.054720,-0.734720,-0.739840,889.575920,209.983920,367.846880,4.030640,-1.528320,-2.554880
3,2019-05-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-12.088240,-0.843776,-0.587776,-0.591872,1385.292736,-207.256864,1094.277504,-248.311488,-1.222656,-2.043904
4,2019-06-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-161.248592,-0.675021,-0.470221,-0.473498,-10.593811,-441.325491,541.176003,-531.961190,-0.978125,-1.635123
5,2019-07-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-153.010874,-0.540017,-0.376177,-0.378798,1849.880951,74.623607,347.018803,17.101048,-0.782500,-1.308099
6,2019-08-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,78.657301,-0.432013,-0.300941,-0.303038,355.266761,-47.199114,292.203042,-100.461162,-0.626000,-1.046479
7,2019-09-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,117.059841,-0.345611,-0.240753,-0.242431,794.665409,48.454709,103.466434,-37.928929,-0.500800,-0.837183
8,2019-10-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,285.957873,-0.276489,-0.192602,0.962055,3279.384327,21.087767,1579.123147,171.948856,-0.400640,-0.669746
9,2019-11-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,437.584298,-0.221191,-0.154082,79.349644,4288.041462,86.506213,2044.490518,83.281085,-0.320512,-0.535797


In [51]:
# CHeck the stationarity of the MAIN drug and make diff - target_morion_id

diff_level = 1
final_stationary_column_name = f'morion_{target_morion_id}_diff_{diff_level}'

while True:
    if check_statio(m_data, final_stationary_column_name):
        print('diff_level: ' + str(diff_level))
        break
    else:
        diff_level += 1
        final_stationary_column_name = f'morion_{target_morion_id}_diff_{diff_level}'
        add_next_differencing_level(m_data, diff_level)
        m_data = m_data.drop(f'morion_{target_morion_id}_diff_{diff_level-1}')

ADF Statistic: -4.957192446935758
p-value: 2.6955270971961126e-05
Stationary
diff_level: 1


In [55]:
# PACF Lags
count_lags = calc_pacf_lags(m_data, final_stationary_column_name)
count_lags

2

In [57]:
# only for target morion - diff order must be equal to count_lags - 1

while diff_level < count_lags:
    print(diff_level, count_lags)
    diff_level +=1
    final_stationary_column_name = f'morion_{target_morion_id}_diff_{diff_level}'
    print(final_stationary_column_name)
    add_next_differencing_level(m_data, target_morion_id, diff_level)
    print(f'morion_{target_morion_id}_diff_{diff_level-1}')
    m_data = m_data.drop([f'morion_{target_morion_id}_diff_{diff_level-1}'], axis=1)


# m_data.columns
# diff_level, count_lags

1 2
morion_112504_diff_2
morion_112504_diff_1


In [59]:
m_data.columns

Index(['month', 'atc_code_5_normalized', 'brand_normalized', 'inn_normalized',
       'nfc_1_normalized', 'strength_unit_normalized',
       'drug_size_unit_normalized', 'form_normalized',
       'strength_amount_normalized', 'pack_size_normalized',
       'drug_size_amount_normalized', 'morion_1479_diff_1',
       'morion_20731_diff_1', 'morion_71569_diff_1', 'morion_71570_diff_1',
       'morion_74821_diff_1', 'morion_75666_diff_1', 'morion_75912_diff_1',
       'morion_75913_diff_1', 'morion_75916_diff_1', 'morion_75917_diff_1',
       'morion_75945_diff_1', 'morion_80191_diff_1', 'morion_80193_diff_1',
       'morion_101789_diff_1', 'morion_112506_diff_1', 'morion_115212_diff_1',
       'morion_115213_diff_1', 'morion_115215_diff_1', 'morion_115216_diff_1',
       'morion_141381_diff_1', 'morion_141382_diff_1', 'morion_160194_diff_1',
       'morion_160195_diff_1', 'morion_289655_diff_1', 'morion_289656_diff_1',
       'morion_289658_diff_1', 'morion_298382_diff_1', 'morion_305988_

In [61]:
m_data[final_stationary_column_name]

0              NaN
1      3669.348000
2      1509.358400
3      2880.038720
4     -3658.435024
5      1151.445981
6      -768.921215
7     -1482.214972
8      2539.814022
9       817.843218
10      475.294574
11    -2860.472341
12     2227.942127
13    -3034.154298
14   -10317.975438
15     4007.783649
16     1700.928919
17     1362.467136
18     -268.614292
19     2225.318567
20     1208.766853
21      -10.282517
22     2969.585986
23    -3805.437211
24      848.848231
25     3614.638585
26    -4417.621132
27    -1007.786906
28      211.398475
29    -2229.637220
30    -1475.873776
31     2055.588979
32     1263.557184
33    -1358.032253
34      676.528197
35      522.874558
36     3757.441646
37     5756.553317
38   -16642.621346
39     5596.660923
40    -5034.743262
41     -712.914609
42     2712.104313
43     3705.035450
44     6815.714360
45    -3135.826512
46      665.704790
47    -4315.234168
48     1848.844666
49    10675.363733
50   -11729.419014
51     1468.800789
52    -2099.

In [63]:
# Add the lag columns according to the cunt_lags 

for i in range(1, count_lags + 1):
    m_data[f'ar_lag_{i}'] = m_data[final_stationary_column_name].shift(i)
    


In [65]:
m_data.shape

(59, 55)

In [67]:
m_data.columns

Index(['month', 'atc_code_5_normalized', 'brand_normalized', 'inn_normalized',
       'nfc_1_normalized', 'strength_unit_normalized',
       'drug_size_unit_normalized', 'form_normalized',
       'strength_amount_normalized', 'pack_size_normalized',
       'drug_size_amount_normalized', 'morion_1479_diff_1',
       'morion_20731_diff_1', 'morion_71569_diff_1', 'morion_71570_diff_1',
       'morion_74821_diff_1', 'morion_75666_diff_1', 'morion_75912_diff_1',
       'morion_75913_diff_1', 'morion_75916_diff_1', 'morion_75917_diff_1',
       'morion_75945_diff_1', 'morion_80191_diff_1', 'morion_80193_diff_1',
       'morion_101789_diff_1', 'morion_112506_diff_1', 'morion_115212_diff_1',
       'morion_115213_diff_1', 'morion_115215_diff_1', 'morion_115216_diff_1',
       'morion_141381_diff_1', 'morion_141382_diff_1', 'morion_160194_diff_1',
       'morion_160195_diff_1', 'morion_289655_diff_1', 'morion_289656_diff_1',
       'morion_289658_diff_1', 'morion_298382_diff_1', 'morion_305988_

In [69]:
morion_columns = [col for col in m_data.columns if 'morion' in col]
filtered_columns = [col for col in morion_columns if not col.endswith(f'_{count_lags}')]
filtered_columns

['morion_1479_diff_1',
 'morion_20731_diff_1',
 'morion_71569_diff_1',
 'morion_71570_diff_1',
 'morion_74821_diff_1',
 'morion_75666_diff_1',
 'morion_75912_diff_1',
 'morion_75913_diff_1',
 'morion_75916_diff_1',
 'morion_75917_diff_1',
 'morion_75945_diff_1',
 'morion_80191_diff_1',
 'morion_80193_diff_1',
 'morion_101789_diff_1',
 'morion_112506_diff_1',
 'morion_115212_diff_1',
 'morion_115213_diff_1',
 'morion_115215_diff_1',
 'morion_115216_diff_1',
 'morion_141381_diff_1',
 'morion_141382_diff_1',
 'morion_160194_diff_1',
 'morion_160195_diff_1',
 'morion_289655_diff_1',
 'morion_289656_diff_1',
 'morion_289658_diff_1',
 'morion_298382_diff_1',
 'morion_305988_diff_1',
 'morion_305993_diff_1',
 'morion_305998_diff_1',
 'morion_328377_diff_1',
 'morion_328378_diff_1',
 'morion_378561_diff_1',
 'morion_378563_diff_1',
 'morion_378565_diff_1',
 'morion_444531_diff_1',
 'morion_444532_diff_1',
 'morion_444535_diff_1',
 'morion_444536_diff_1',
 'morion_614190_diff_1',
 'morion_61419

In [71]:
diff_level = int(filtered_columns[0].split('_')[-1])
diff_level

1

In [73]:
count_lags

2

In [75]:
# The rest of the drug will be features - but they have to be the same level of differencing 

for i in range(diff_level, count_lags-1):
    for col in filtered_columns:
        morion = col.split('_')[1]
        diff_level = int(col[-1:].split('_')[-1])
        diff_level += 1
        add_next_differencing_level(m_data, morion, diff_level)
        m_data = m_data.drop([f'morion_{morion}_diff_{diff_level-1}'], axis=1)
        

In [77]:
m_data

,month,atc_code_5_normalized,brand_normalized,inn_normalized,nfc_1_normalized,strength_unit_normalized,drug_size_unit_normalized,form_normalized,strength_amount_normalized,pack_size_normalized,...,morion_378565_diff_1,morion_444531_diff_1,morion_444532_diff_1,morion_444535_diff_1,morion_444536_diff_1,morion_614190_diff_1,morion_614194_diff_1,morion_112504_diff_2,ar_lag_1,ar_lag_2
0,2019-02-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-1.156000,-284.052000,545.258000,73.952000,328.546000,-2.388000,-3.992000,NaN,NaN,NaN
1,2019-03-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-0.924800,-26.217600,914.402400,-212.266400,349.360800,-1.910400,-3.193600,3669.348000,NaN,NaN
2,2019-04-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-0.739840,889.575920,209.983920,367.846880,4.030640,-1.528320,-2.554880,1509.358400,3669.348000,NaN
3,2019-05-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-0.591872,1385.292736,-207.256864,1094.277504,-248.311488,-1.222656,-2.043904,2880.038720,1509.358400,3669.348000
4,2019-06-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-0.473498,-10.593811,-441.325491,541.176003,-531.961190,-0.978125,-1.635123,-3658.435024,2880.038720,1509.358400
5,2019-07-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-0.378798,1849.880951,74.623607,347.018803,17.101048,-0.782500,-1.308099,1151.445981,-3658.435024,2880.038720
6,2019-08-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-0.303038,355.266761,-47.199114,292.203042,-100.461162,-0.626000,-1.046479,-768.921215,1151.445981,-3658.435024
7,2019-09-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,-0.242431,794.665409,48.454709,103.466434,-37.928929,-0.500800,-0.837183,-1482.214972,-768.921215,1151.445981
8,2019-10-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,0.962055,3279.384327,21.087767,1579.123147,171.948856,-0.400640,-0.669746,2539.814022,-1482.214972,-768.921215
9,2019-11-01,0.111111,0.366972,0.315789,0.375,0.4,1.0,1.0,0.0025,0.243697,...,79.349644,4288.041462,86.506213,2044.490518,83.281085,-0.320512,-0.535797,817.843218,2539.814022,-1482.214972


In [79]:
m_data = m_data.dropna()

In [81]:
m_data.shape

(56, 55)

In [83]:
m_data.to_csv((r"C:\Users\Madnessity\Desktop\_teva_sales_small_diff_pacf_lags.csv')

In [85]:
m_data.to_excel(r"C:\Users\Madnessity\Desktop\_teva_sales_small_diff_pacf_lags.xlsx")